In [1]:
"""Loading data with LUSID Python Tools

Demonstrates how to load portfolios, instruments, holdings, and transactions.

Attributes
----------
instruments
transactions
portfolios
cocoon
"""

'Loading data with LUSID Python Tools\n\nDemonstrates how to load portfolios, instruments, holdings, and transactions.\n\nAttributes\n----------\ninstruments\ntransactions\nportfolios\ncocoon\n'

# Import your Libraries

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import requests
import os
import lusid
import lusidtools.cocoon.cocoon as cocoon
import uuid

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

scope = 'my_test_scope_12'

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.3734.0


# 1) Create Portfolios

## a) Import portfolio data

In [2]:
positions_file = pd.read_csv('./data/Sample_Positions.csv')
positions_file.head()

FundCode Effective Date Prime Broker Local Currency Code  Quantity  \
0  Portfolio-Z     02/07/2019          UBS                 GBP      2500   
1  Portfolio-Z     01/01/2019         CITI                 USD     10000   
2  Portfolio-Z     01/12/2018          UBS                 USD     26598   
3  Portfolio-X     05/02/2019         BAML                 GBX    942354   
4  Portfolio-X     24/11/2018           MS                 USD     95421   

   Local Price  Local Market Value  Base Market Value  \
0        56.76              141900             141900   
1       208.40             2084000            2084000   
2      1989.50            52916721           52916721   
3       200.50           188941977          188941977   
4       230.96            22038434           22038434   

       Security Description ISIN Security Identifier  \
0  Lloyds Banking Group PLC             GB0008706128   
1                 Apple Inc             US0378331005   
2                    Amazon             US0231351067   
3                 Sainsbury            XS1139087933    
4                 Tesla Inc             US88160R1014   

  SEDOL Security Identifier Buy_Currency  
0                    870612          GBP  
1                  0865985           USD  
2                    906866          USD  
3                   B019KW7          GBP  
4                   BSJC712          USD

## b) Apply data transformations specific to your data

In [3]:
portfolios_data = pd.DataFrame(positions_file['FundCode'])
portfolios_data['display_name'] = portfolios_data['FundCode'].apply(lambda x: "Fund {}".format(x))
portfolios_data['created'] = "2010-10-09T08:00:00Z"
portfolios_data['base_currency'] = 'GBP'
portfolios_data['description'] = portfolios_data['FundCode'].apply(lambda x: "The fund with the fund code {}".format(x))
portfolios_data.head()

FundCode      display_name               created base_currency  \
0  Portfolio-Z  Fund Portfolio-Z  2010-10-09T08:00:00Z           GBP   
1  Portfolio-Z  Fund Portfolio-Z  2010-10-09T08:00:00Z           GBP   
2  Portfolio-Z  Fund Portfolio-Z  2010-10-09T08:00:00Z           GBP   
3  Portfolio-X  Fund Portfolio-X  2010-10-09T08:00:00Z           GBP   
4  Portfolio-X  Fund Portfolio-X  2010-10-09T08:00:00Z           GBP   

                               description  
0  The fund with the fund code Portfolio-Z  
1  The fund with the fund code Portfolio-Z  
2  The fund with the fund code Portfolio-Z  
3  The fund with the fund code Portfolio-X  
4  The fund with the fund code Portfolio-X

## c) Set your portfolio field mapping

In [4]:
portfolio_mapping_required = {
  'code': 'FundCode',
  'display_name': 'display_name',
  'base_currency': 'base_currency'
}

portfolio_mapping_optional = {
  'description': 'description',
  'accounting_method': None,
  'created': 'created'
}

## d) Create your portfolios

In [5]:
responses = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=portfolios_data, 
    mapping_required=portfolio_mapping_required, 
    mapping_optional=portfolio_mapping_optional,
    file_type='portfolios')

for response in responses['portfolios']['success']:
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: my_test_scope_12
Code: Portfolio-Z
Portfolio Effective From: 2010-10-09 08:00:00+00:00
Portfolio Created On: 2020-01-02 15:56:23.905020+00:00

Portfolio Created
Scope: my_test_scope_12
Code: Portfolio-X
Portfolio Effective From: 2010-10-09 08:00:00+00:00
Portfolio Created On: 2020-01-02 15:56:23.890817+00:00



# 2) Load your instrument universe

## a) Import instrument data

In [6]:
positions_file

FundCode Effective Date Prime Broker Local Currency Code  Quantity  \
0  Portfolio-Z     02/07/2019          UBS                 GBP      2500   
1  Portfolio-Z     01/01/2019         CITI                 USD     10000   
2  Portfolio-Z     01/12/2018          UBS                 USD     26598   
3  Portfolio-X     05/02/2019         BAML                 GBX    942354   
4  Portfolio-X     24/11/2018           MS                 USD     95421   

   Local Price  Local Market Value  Base Market Value  \
0        56.76              141900             141900   
1       208.40             2084000            2084000   
2      1989.50            52916721           52916721   
3       200.50           188941977          188941977   
4       230.96            22038434           22038434   

       Security Description ISIN Security Identifier  \
0  Lloyds Banking Group PLC             GB0008706128   
1                 Apple Inc             US0378331005   
2                    Amazon             US0231351067   
3                 Sainsbury            XS1139087933    
4                 Tesla Inc             US88160R1014   

  SEDOL Security Identifier Buy_Currency  
0                    870612          GBP  
1                  0865985           USD  
2                    906866          USD  
3                   B019KW7          GBP  
4                   BSJC712          USD

## b) Apply data transformations specific to your data

In [7]:
instruments_data = pd.DataFrame(
    positions_file.loc[:, [
        'Local Currency Code', 
        'Security Description',
        'ISIN Security Identifier',
        'SEDOL Security Identifier']]
    )

instruments_data

Local Currency Code      Security Description ISIN Security Identifier  \
0                 GBP  Lloyds Banking Group PLC             GB0008706128   
1                 USD                 Apple Inc             US0378331005   
2                 USD                    Amazon             US0231351067   
3                 GBX                 Sainsbury            XS1139087933    
4                 USD                 Tesla Inc             US88160R1014   

  SEDOL Security Identifier  
0                    870612  
1                  0865985   
2                    906866  
3                   B019KW7  
4                   BSJC712

## c) Set your instruments field mapping

In [8]:
instrument_mapping_required = {
  'name': 'Security Description'
}

instrument_identifier_mapping = {
    'ClientInternal': 'SEDOL Security Identifier',
    'Sedol': 'SEDOL Security Identifier',
    'Isin': 'ISIN Security Identifier'
}

instrument_mapping_optional = {
  'definition': None
}

## d) Upsert your instruments

In [9]:
response = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    data_frame=instruments_data, 
    identifier_mapping=instrument_identifier_mapping, 
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type='instruments',
    scope='InstrumentProperties001')

prettyprint.instrument_response(response['instruments']['success'][0])

Instruments Successfully Upserted: 


Instrument ClientInternal ID LUSID Instrument ID
0   ClientInternal: 906866            906866       LUID_9CSM3NHY
1  ClientInternal: 0865985           0865985       LUID_NBWB57SI
2  ClientInternal: B019KW7           B019KW7       LUID_4BC54H2H
3  ClientInternal: BSJC712           BSJC712       LUID_AGYGGMBS
4   ClientInternal: 870612            870612       LUID_DRSVHB3Q

# 3) Set your holdings

## a) Import holdings data

In [10]:
holdings_file = positions_file.copy(deep=True)
holdings_file

FundCode Effective Date Prime Broker Local Currency Code  Quantity  \
0  Portfolio-Z     02/07/2019          UBS                 GBP      2500   
1  Portfolio-Z     01/01/2019         CITI                 USD     10000   
2  Portfolio-Z     01/12/2018          UBS                 USD     26598   
3  Portfolio-X     05/02/2019         BAML                 GBX    942354   
4  Portfolio-X     24/11/2018           MS                 USD     95421   

   Local Price  Local Market Value  Base Market Value  \
0        56.76              141900             141900   
1       208.40             2084000            2084000   
2      1989.50            52916721           52916721   
3       200.50           188941977          188941977   
4       230.96            22038434           22038434   

       Security Description ISIN Security Identifier  \
0  Lloyds Banking Group PLC             GB0008706128   
1                 Apple Inc             US0378331005   
2                    Amazon             US0231351067   
3                 Sainsbury            XS1139087933    
4                 Tesla Inc             US88160R1014   

  SEDOL Security Identifier Buy_Currency  
0                    870612          GBP  
1                  0865985           USD  
2                    906866          USD  
3                   B019KW7          GBP  
4                   BSJC712          USD

## b) Apply data transformations specific to your data

In [11]:
holdings_file['is_cash_with_currency'] = np.NaN

# Convert amounts in Pence to be in Pounds
for column in ["Local Price", "Local Market Value",  "Base Market Value"]:
    holdings_file[column] = holdings_file.apply(
        lambda x: getattr(x,column)/100 if getattr(x, "Local Currency Code") == "GBX" else getattr(x,column), axis=1)
    
holdings_file['Local Currency Code'] = holdings_file['Local Currency Code'].apply(
    lambda x: "GBP" if x == "GBX" else x)
holdings_file.head()

FundCode Effective Date Prime Broker Local Currency Code  Quantity  \
0  Portfolio-Z     02/07/2019          UBS                 GBP      2500   
1  Portfolio-Z     01/01/2019         CITI                 USD     10000   
2  Portfolio-Z     01/12/2018          UBS                 USD     26598   
3  Portfolio-X     05/02/2019         BAML                 GBP    942354   
4  Portfolio-X     24/11/2018           MS                 USD     95421   

   Local Price  Local Market Value  Base Market Value  \
0        56.76           141900.00          141900.00   
1       208.40          2084000.00         2084000.00   
2      1989.50         52916721.00        52916721.00   
3         2.00          1889419.77         1889419.77   
4       230.96         22038434.00        22038434.00   

       Security Description ISIN Security Identifier  \
0  Lloyds Banking Group PLC             GB0008706128   
1                 Apple Inc             US0378331005   
2                    Amazon             US0231351067   
3                 Sainsbury            XS1139087933    
4                 Tesla Inc             US88160R1014   

  SEDOL Security Identifier Buy_Currency  is_cash_with_currency  
0                    870612          GBP                    nan  
1                  0865985           USD                    nan  
2                    906866          USD                    nan  
3                   B019KW7          GBP                    nan  
4                   BSJC712          USD                    nan

## c) Set your holdings field mapping

In [12]:
holding_mapping_required = {
  'code': 'FundCode',
  'effective_at': 'Effective Date',
  'tax_lots.units': 'Quantity'
}

holding_identifier_mapping = {
  'Isin': 'ISIN Security Identifier',
  'Sedol': 'SEDOL Security Identifier',
  'ClientInternal': 'SEDOL Security Identifier',
  'Currency': 'is_cash_with_currency'
}

holding_mapping_optional= {
  'tax_lots.cost.amount': None,
  'tax_lots.cost.currency': "Local Currency Code",
  'tax_lots.portfolio_cost': None,
  'tax_lots.price': None,
  'tax_lots.purchase_date': None,
  'tax_lots.settlement_date': None
}

## d) Add your holdings

In [13]:
responses = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=holdings_file,
    mapping_required=holding_mapping_required, 
    mapping_optional=holding_mapping_optional,
    identifier_mapping=holding_identifier_mapping,
    file_type='holding')

for response in responses['holdings']['success']:
    print ('\n')
    prettyprint.adjust_holdings_response(response, scope, response.href.split("/")[7])



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2020-01-02 15:56:23.905020+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2020-01-02 15:56:23.905020+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2020-01-02 15:56:23.905020+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-X
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00
Adjusted Holdings Created On: 2020-01-02 15:56:23.890817+00:00



Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_12
Code: Portfolio-X
Adjusted Holdings Effective From: 2010-10-09 08:00:00+00:00

In [14]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code="Portfolio-X",
    property_keys=["Instrument/default/Name"])

holdings = prettyprint.get_holdings_df(response)
holdings

_instrument_uid _holding_type    _units  _settled_units _transaction  \
0   LUID_4BC54H2H             P 942354.00       942354.00         None   

  discriminator  cost.amount cost.currency  cost_portfolio_ccy.amount  \
0          None         0.00           GBP                       0.00   

  Instrument/default/Name  
0               Sainsbury

In [15]:
holdings.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
_instrument_uid              1 non-null object
_holding_type                1 non-null object
_units                       1 non-null float64
_settled_units               1 non-null float64
_transaction                 0 non-null object
discriminator                0 non-null object
cost.amount                  1 non-null float64
cost.currency                1 non-null object
cost_portfolio_ccy.amount    1 non-null float64
Instrument/default/Name      1 non-null object
dtypes: float64(4), object(6)
memory usage: 208.0+ bytes


# 4) Add your transactions

## a) Import transactions data

In [16]:
transactions_data = positions_file.copy(deep=True)
transactions_data

FundCode Effective Date Prime Broker Local Currency Code  Quantity  \
0  Portfolio-Z     02/07/2019          UBS                 GBP      2500   
1  Portfolio-Z     01/01/2019         CITI                 USD     10000   
2  Portfolio-Z     01/12/2018          UBS                 USD     26598   
3  Portfolio-X     05/02/2019         BAML                 GBX    942354   
4  Portfolio-X     24/11/2018           MS                 USD     95421   

   Local Price  Local Market Value  Base Market Value  \
0        56.76              141900             141900   
1       208.40             2084000            2084000   
2      1989.50            52916721           52916721   
3       200.50           188941977          188941977   
4       230.96            22038434           22038434   

       Security Description ISIN Security Identifier  \
0  Lloyds Banking Group PLC             GB0008706128   
1                 Apple Inc             US0378331005   
2                    Amazon             US0231351067   
3                 Sainsbury            XS1139087933    
4                 Tesla Inc             US88160R1014   

  SEDOL Security Identifier Buy_Currency  
0                    870612          GBP  
1                  0865985           USD  
2                    906866          USD  
3                   B019KW7          GBP  
4                   BSJC712          USD

## b) Apply data transformations specific to your data

In [17]:
transactions_data['transaction_type'] = 'FundsIn'
transactions_data['exchange_rate'] = 1
transactions_data['is_cash_with_currency'] = np.NaN

# Convert amounts in Pence to be in Pounds
for column in ["Local Price", "Local Market Value",  "Base Market Value"]:
    transactions_data[column] = transactions_data.apply(
        lambda x: getattr(x,column)/100 if getattr(x, "Local Currency Code") == "GBX" else getattr(x,column), axis=1)
transactions_data['Local Currency Code'] = transactions_data['Local Currency Code'].apply(
    lambda x: "GBP" if x == "GBX" else x)

transactions_data['transaction_id'] = [str(uuid.uuid4()) for _ in range(len(transactions_data.index))]
transactions_data['Effective Date'] = pd.to_datetime(
    transactions_data['Effective Date'], format="%d/%m/%Y").apply(lambda x: pytz.utc.localize(x))
transactions_data.head()

FundCode            Effective Date Prime Broker Local Currency Code  \
0  Portfolio-Z 2019-07-02 00:00:00+00:00          UBS                 GBP   
1  Portfolio-Z 2019-01-01 00:00:00+00:00         CITI                 USD   
2  Portfolio-Z 2018-12-01 00:00:00+00:00          UBS                 USD   
3  Portfolio-X 2019-02-05 00:00:00+00:00         BAML                 GBP   
4  Portfolio-X 2018-11-24 00:00:00+00:00           MS                 USD   

   Quantity  Local Price  Local Market Value  Base Market Value  \
0      2500        56.76           141900.00          141900.00   
1     10000       208.40          2084000.00         2084000.00   
2     26598      1989.50         52916721.00        52916721.00   
3    942354         2.00          1889419.77         1889419.77   
4     95421       230.96         22038434.00        22038434.00   

       Security Description ISIN Security Identifier  \
0  Lloyds Banking Group PLC             GB0008706128   
1                 Apple Inc             US0378331005   
2                    Amazon             US0231351067   
3                 Sainsbury            XS1139087933    
4                 Tesla Inc             US88160R1014   

  SEDOL Security Identifier Buy_Currency transaction_type  exchange_rate  \
0                    870612          GBP          FundsIn              1   
1                  0865985           USD          FundsIn              1   
2                    906866          USD          FundsIn              1   
3                   B019KW7          GBP          FundsIn              1   
4                   BSJC712          USD          FundsIn              1   

   is_cash_with_currency                        transaction_id  
0                    nan  128cda85-71a9-4a2c-832b-9e5aa75e7854  
1                    nan  501fb1a9-6239-4b37-852a-e3e16e3f8c94  
2                    nan  b9e5bb39-7148-4f96-a8bd-36ef75566a31  
3                    nan  444d960e-90ac-43ab-9648-3271aab5ec8a  
4                    nan  246f18ec-7a2a-488a-99d9-e89cd90a4166

## c) Set your transactions field mapping

In [18]:
transaction_field_mapping_required = {
    "code": "FundCode",
    "transaction_id": "transaction_id",
    "type": 'transaction_type',
    "transaction_date": 'Effective Date',
    "settlement_date": 'Effective Date',
    "units": "Quantity",
    "transaction_price.price": "Local Price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "Local Market Value",
    "total_consideration.currency": "Local Currency Code",
    "transaction_currency": "Local Currency Code"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "exchange_rate",
    "source": "$Client"
}

transaction_identifier_mapping = {
  'Isin': 'ISIN Security Identifier',
  'Sedol': 'SEDOL Security Identifier',
  'ClientInternal': 'SEDOL Security Identifier',
  "Currency": 'is_cash_with_currency'
    
}

## d) Add your transactions

In [19]:
responses = cocoon.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=transactions_data,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    identifier_mapping=transaction_identifier_mapping,
    file_type='transaction')

for response in responses['transactions']['success']:
    print ('\n')
    prettyprint.transactions_response(response, scope, response.href.split("/")[7])



Transactions Successfully Upserted into Portfolio
Scope: my_test_scope_12
Code: Portfolio-Z
Transactions Effective From: 2019-07-02 00:00:00+00:00
Transactions Created On: 2020-01-02 15:56:27.133274+00:00



Transactions Successfully Upserted into Portfolio
Scope: my_test_scope_12
Code: Portfolio-X
Transactions Effective From: 2019-02-05 00:00:00+00:00
Transactions Created On: 2020-01-02 15:56:27.125480+00:00



In [20]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code="Portfolio-X",
    property_keys=["Instrument/default/Name"])

holdings = prettyprint.get_holdings_df(response)
holdings

_instrument_uid _holding_type    _units  _settled_units _transaction  \
0   LUID_4BC54H2H             P 942354.00       942354.00         None   

  discriminator  cost.amount cost.currency  cost_portfolio_ccy.amount  \
0          None         0.00           GBP                       0.00   

  Instrument/default/Name  
0               Sainsbury